In [ ]:
import torch

from occhio.autoencoder import TiedLinearRelu
from occhio.distributions import SparseUniform
from occhio.model_grid import ModelGrid, Axis
from occhio.toy_model import ToyModel
from occhio.visualization import (
    plot_embedding,
    plot_phase_change,
    plot_representation,
    plot_geometry,
)

In [ ]:
device = "mps"

# Background & Motivation

In [ ]:
N_FEATURES = 5
N_HIDDEN = 2

In [ ]:
def create_model(params):
    generator = torch.Generator(device).manual_seed(7)

    return ToyModel(
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, generator=generator),
        distribution=SparseUniform(
            N_FEATURES, p_active=1 - params["Sparsity"], generator=generator
        ),
        importances=0.9 ** torch.arange(N_FEATURES),
        device=device,
    )

In [ ]:
grid = ModelGrid(
    create_model,
    axes=[Axis(label="Sparsity", values=[0.0, 0.8, 0.9])],
)

In [ ]:
grid.fit()

In [ ]:
plot_embedding(grid)

# Demonstrating Superposition

## 20 x 5

In [ ]:
N_FEATURES = 20
N_HIDDEN = 5

In [ ]:
def create_model(params):
    generator = torch.Generator(device).manual_seed(199)

    return ToyModel(
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, generator=generator),
        distribution=SparseUniform(
            N_FEATURES, p_active=params["Density"], generator=generator
        ),
        importances=0.7 ** torch.arange(N_FEATURES),
        device=device,
    )

In [ ]:
grid = ModelGrid(
    create_model,
    axes=[Axis(label="Density", values=[1, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001])],
)

In [ ]:
grid.fit()

In [ ]:
plot_representation(grid)

## 80 x 20


In [ ]:
N_FEATURES = 20
N_HIDDEN = 5

In [ ]:
def create_model(params):
    generator = torch.Generator(device).manual_seed(199)

    return ToyModel(
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, generator=generator),
        distribution=SparseUniform(
            N_FEATURES, p_active=params["Density"], generator=generator
        ),
        importances=0.9 ** torch.arange(N_FEATURES),
        device=device,
    )

In [ ]:
grid = ModelGrid(
    create_model,
    axes=[Axis(label="Density", values=[1, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001])],
)

In [ ]:
grid.fit()

In [ ]:
plot_representation(grid)

# Superposition as a Phase Change

## 2 Features

In [ ]:
N_FEATURES = 2
N_HIDDEN = 1
EXPERIMENT_SIZE = 21

In [ ]:
def create_model(params):
    generator = torch.Generator(device).manual_seed(8)

    return ToyModel(
        distribution=SparseUniform(
            N_FEATURES, p_active=params["Density"], generator=generator
        ),
        importances=[1, params["Relative Importance"]],
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, generator=generator),
        device=device,
    )

In [ ]:
grid = ModelGrid(
    create_model,
    axes=[
        Axis(
            label="Relative Importance", values=torch.logspace(-1, 1, EXPERIMENT_SIZE)
        ),
        Axis(label="Density", values=torch.logspace(-2, 0, EXPERIMENT_SIZE)),
    ],
)

In [ ]:
grid.fit()

In [ ]:
plot_phase_change(grid, tracked_feature=1)

## 3 Features

In [ ]:
N_FEATURES = 3
N_HIDDEN = 2
EXPERIMENT_SIZE = 21

In [ ]:
def create_model(params):
    generator = torch.Generator(device).manual_seed(8)

    return ToyModel(
        distribution=SparseUniform(
            N_FEATURES, p_active=params["Density"], generator=generator
        ),
        importances=[1, 1, params["Relative Importance"]],
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, generator=generator),
        device=device,
    )

In [ ]:
grid = ModelGrid(
    create_model,
    axes=[
        Axis(
            label="Relative Importance", values=torch.logspace(-1, 1, EXPERIMENT_SIZE)
        ),
        Axis(label="Density", values=torch.logspace(-2, 0, EXPERIMENT_SIZE)),
    ],
)

In [ ]:
grid.fit()

In [ ]:
plot_phase_change(grid, tracked_feature=2)

# The Geometry of Superposition

In [ ]:
N_FEATURES = 400
N_HIDDEN = 30

In [ ]:
def create_model(params):
    generator = torch.Generator(device=device).manual_seed(8)

    return ToyModel(
        distribution=SparseUniform(
            N_FEATURES, p_active=params["Density"], device=device, generator=generator
        ),
        ae=TiedLinearRelu(N_FEATURES, N_HIDDEN, device=device, generator=generator),
    )

In [ ]:
grid = ModelGrid(
    create_model, axes=[Axis(label="Density", values=10 ** torch.linspace(0, -1, 50))]
)

In [ ]:
grid.fit(100)

In [ ]:
plot_geometry(grid)